Import


In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, average_precision_score


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Load Dataset (Keras)

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "/content/drive/MyDrive/final_dataset/train"
val_dir   = "/content/drive/MyDrive/final_dataset/val"
test_dir  = "/content/drive/MyDrive/final_dataset/test"

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")
val_gen   = datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")
test_gen  = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False)

NUM_CLASSES = train_gen.num_classes


Found 372 images belonging to 4 classes.
Found 79 images belonging to 4 classes.
Found 88 images belonging to 4 classes.


Model Builder Function

In [5]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(base_model):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(NUM_CLASSES, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(
        optimizer=Adam(0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


Train + Record Time

In [6]:
def train_model(model, name):
    start = time.time()
    history = model.fit(train_gen, validation_data=val_gen, epochs=50)
    end = time.time()
    print(f"{name} Training Time: {end-start:.2f} seconds")
    return history, end-start


Train All Models

In [7]:
from tensorflow.keras.applications import ResNet50, DenseNet121, MobileNetV3Large

models = {
    "ResNet50": ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3)),
    "DenseNet121": DenseNet121(weights="imagenet", include_top=False, input_shape=(224,224,3)),
    "MobileNetV3": MobileNetV3Large(weights="imagenet", include_top=False, input_shape=(224,224,3))
}

histories = {}
times = {}

for name, base in models.items():
    model = build_model(base)
    hist, t = train_model(model, name)
    histories[name] = hist
    times[name] = t


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 145s 11s/step - accuracy: 0.1945 - loss: 1.6738 - val_accuracy: 0.2911 - val_loss: 1.4044
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 208ms/step - accuracy: 0.2505 - loss: 1.4142 - val_accuracy: 0.2785 - val_loss: 1.3541
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - accuracy: 0.3157 - loss: 1.3537 - val_accuracy: 0.5443 - val_loss: 1.3110
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.4195 - loss: 1.3473 - val_accuracy: 0.4304 - val_loss: 1.2891
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 178ms/step - accuracy: 0.3606 - loss: 1.3274 - val_accuracy: 0.4810 - val_loss: 1.2708
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.4342 - loss: 1.3116 - val_accuracy: 0.5696 - val_loss: 1.2509
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 203ms/step - accuracy: 0.5057 - loss: 1.2822 - val_accuracy: 0.5949 - val_loss: 1.2368
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 214ms/step - accuracy: 0.5106 - loss: 1.2764 - val_accuracy: 0.

Fine-Tuning (Hyperparameter Tuning)

In [8]:
for layer in model.layers[-10:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_gen, validation_data=val_gen, epochs=20)


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.1765 - loss: 10.6935 - val_accuracy: 0.4557 - val_loss: 1.2678
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.1708 - loss: 8.4778 - val_accuracy: 0.4051 - val_loss: 1.2762
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.1859 - loss: 8.0959 - val_accuracy: 0.3797 - val_loss: 1.2823
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.1860 - loss: 7.2715 - val_accuracy: 0.3291 - val_loss: 1.2903
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.2105 - loss: 7.0588 - val_accuracy: 0.3165 - val_loss: 1.2989
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.1792 - loss: 6.0306 - val_accuracy: 0.3165 - val_loss: 1.3069
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.1605 - loss: 6.4838 - val_accuracy: 0.3291 - val_loss: 1.3140
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step - accuracy: 0.1657 - loss: 5.7409 - val_accuracy: 0.3

Evaluation (Accuracy + mAP)

In [9]:
pred_probs = model.predict(test_gen)
pred_classes = np.argmax(pred_probs, axis=1)
true_classes = test_gen.classes

acc = accuracy_score(true_classes, pred_classes)
y_true = tf.keras.utils.to_categorical(true_classes, NUM_CLASSES)
mAP = average_precision_score(y_true, pred_probs, average="macro")

print("Test Accuracy:", acc)
print("mAP:", mAP)


3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 11s/step
Test Accuracy: 0.42045454545454547
mAP: 0.4645410063240163
